In [297]:
import pandas as pd 
import collections
import itertools

In [298]:
movie_bd = pd.read_csv('movie_bd_v5.csv') #Проверим, что файл читается и запишем данные в датафрейм.

# Предобработка

In [299]:
answers = {} # Создадим словарь для ответов 

#Посмотрим на весь датасет.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(movie_bd.info())
    display(movie_bd.describe())
    display(movie_bd.describe(include = ['object']))
    display(movie_bd) 
    
#Добавим новый столбец - прибыль ('profit') расчитывающий прибыль по формуле:
#Прибыль = сборы - бюджет (profit = revenue - budget)
movie_bd['profit'] = movie_bd['revenue'] - movie_bd['budget']

#Добавим новый столбец - месяц ('month') для этого разделим столбец release_date цифры по знаку '/' с помощью метода split 
#и возьмем первое значение
movie_bd['release_date'] = movie_bd['release_date'].apply(lambda x: x.split('/')) 
movie_bd['month']=movie_bd['release_date'].apply(lambda x: x[0])

#Добавим новый столбец - длина названия ('name_length') и запишем в него количество символов в названии
movie_bd['name_length'] = movie_bd.original_title.apply(lambda x: len(x))
#Разделим production_companies по знаку "|" с помощью метода split 
movie_bd['production_companies_sep'] = movie_bd.production_companies.str.split("|")

#Добавим новый столбец - количество слов в описании ('overview_words') и запишем в него количество слов в названии
movie_bd['overview_words'] = movie_bd.overview.apply(lambda x: len(x.split(' ')))

#разделим значения столбца cast по знаку '|' с помощью метода split
movie_bd['cast_pair'] = movie_bd['cast'].str.split('|')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


None

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


,imdb_id,original_title,cast,director,tagline,overview,genres,production_companies,release_date
count,1889,1889,1889,1889,1889,1889,1889,1889,1889
unique,1889,1887,1878,957,1880,1889,652,1624,1435
top,tt3164256,Fantastic Four,Jennifer Lawrence|Josh Hutcherson|Liam Hemswor...,Steven Soderbergh,Worlds Collide,Brilliant student Jeff Chang has the most impo...,Comedy,New Line Cinema,12/25/2014
freq,1,2,3,13,2,1,141,21,7


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015
5,tt1663202,135000000,532950503,The Revenant,Leonardo DiCaprio|Tom Hardy|Will Poulter|Domhn...,Alejandro GonzÃ¡lez IÃ±Ã¡rritu,"(n. One who has returned, as if from the dead.)","In the 1820s, a frontiersman, Hugh Glass, sets...",156,Western|Drama|Adventure|Thriller,Regency Enterprises|Appian Way|CatchPlay|Anony...,12/25/2015,7.2,2015
6,tt1340138,155000000,440603537,Terminator Genisys,Arnold Schwarzenegger|Jason Clarke|Emilia Clar...,Alan Taylor,Reset the future,"The year is 2029. John Connor, leader of the r...",125,Science Fiction|Action|Thriller|Adventure,Paramount Pictures|Skydance Productions,6/23/2015,5.8,2015
7,tt3659388,108000000,595380321,The Martian,Matt Damon|Jessica Chastain|Kristen Wiig|Jeff ...,Ridley Scott,Bring Him Home,"During a manned mission to Mars, Astronaut Mar...",141,Drama|Adventure|Science Fiction,Twentieth Century Fox Film Corporation|Scott F...,9/30/2015,7.6,2015
8,tt2293640,74000000,1156730962,Minions,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda|Pierre Coffin,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",91,Family|Animation|Adventure|Comedy,Universal Pictures|Illumination Entertainment,6/17/2015,6.5,2015
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [300]:
# В словарь вставляем номер вопроса и ваш ответ на него
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# +

In [301]:
movie_bd[movie_bd.budget == movie_bd.budget.max()] #Найдем строку с максимальным значением в колонке budget

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,"[5, 11, 2011]",6.3,2011,641683000,5,43,"[Walt Disney Pictures, Jerry Bruckheimer Films...",75,"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I..."


# 2. Какой из фильмов самый длительный (в минутах)?

In [302]:
answers['2'] = '2. Gods and Generals (tt0279111)'
# +

In [303]:
movie_bd[movie_bd.runtime == movie_bd.runtime.max()] #Найдем строку с максимальным значением в колонке runtime

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,"[2, 21, 2003]",5.8,2003,-43076064,2,17,"[Turner Pictures, Antietam Filmworks]",48,"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke..."


# 3. Какой из фильмов самый короткий (в минутах)?





In [304]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'
# +

In [305]:
movie_bd[movie_bd.runtime == movie_bd.runtime.min()] #Найдем строку с минимальным значением в колонке runtime

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,"[4, 13, 2011]",6.8,2011,-15540000,4,15,"[Walt Disney Pictures, Walt Disney Animation S...",52,"[Jim Cummings, Travis Oates, Jim Cummings, Bud..."


# 4. Какова средняя длительность фильмов?


In [306]:
answers['4'] = '2. 110'
# +

In [307]:
movie_bd.runtime.mean() # Найдем среднее значение для колонки runtime 

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [308]:
answers['5'] = '1. 107'
# +

In [309]:
movie_bd.runtime.median() # Найдем медиану для колонки runtime 

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [310]:
answers['6'] = '5. Avatar (tt0499549)'
# +

In [311]:
movie_bd[movie_bd.profit == movie_bd.profit.max()] #Найдем строку с максимальным значением в колонке profit

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,"[12, 10, 2009]",7.1,2009,2544505847,12,6,"[Ingenious Film Partners, Twentieth Century Fo...",28,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."


# 7. Какой фильм самый убыточный? 

In [312]:
answers['7'] = '5. The Lone Ranger (tt1210819)'
# +

In [313]:
movie_bd[movie_bd.profit == movie_bd.profit.min()] #Найдем строку с минимальным значением в колонке profit

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,"[7, 3, 2013]",6.0,2013,-165710090,7,15,"[Walt Disney Pictures, Jerry Bruckheimer Films...",68,"[Johnny Depp, Armie Hammer, William Fichtner, ..."


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [314]:
answers['8'] = '1. 1478'
# +

In [315]:
profit_movies = movie_bd[movie_bd.profit > 0] #Сформируем выборку таких фильмов по условию: значение в колонке profit > 0.
len(profit_movies.index) #Подсчитаем количество фильмов выборке

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [316]:
answers['9'] = '4. The Dark Knight (tt0468569)'
# +

In [317]:
movies_2008 = movie_bd.query('release_year == 2008') #Сформируем выборку фильмов по условию: значение в колонке release_year==2008
movies_2008[movies_2008.profit == movies_2008.profit.max()] #Найдем строку с максимальным значением в колонке profit

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,"[7, 16, 2008]",8.1,2008,816921825,7,15,"[DC Comics, Legendary Pictures, Warner Bros., ...",67,"[Christian Bale, Michael Caine, Heath Ledger, ..."


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [318]:
answers['10'] = '5. The Lone Ranger (tt1210819)'
# +

In [319]:
movies_12_14 = movie_bd.query('release_year in ["2012","2013","2014"]') #Сформируем выборку фильмов по условию:значение в колонке release_year in ["2012","2013","2014"]
movies_12_14[movies_12_14.profit == movies_12_14.profit.min()] #Найдем строку с минимальным значением в колонке profit

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,"[7, 3, 2013]",6.0,2013,-165710090,7,15,"[Walt Disney Pictures, Jerry Bruckheimer Films...",68,"[Johnny Depp, Armie Hammer, William Fichtner, ..."


# 11. Какого жанра фильмов больше всего?

In [320]:
answers['11'] = '3. Drama'
# +

In [321]:
collections.Counter(movie_bd['genres'].str.cat(sep = '|').split('|')).most_common(5) #подсчитаем значения genres, разделив по разделителю |

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [322]:
answers['12'] = '1. Drama'
# +

In [323]:
profit_movies = movie_bd[movie_bd.profit > 0] #сформируем выборку таких фильмов по условию: значение в колонке profit > 0.
collections.Counter(profit_movies['genres'].str.cat(sep = '|').split('|')).most_common(5) #подсчитаем значения genres, разделив по разделителю |

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [324]:
answers['13'] = '5. Peter Jackson'
# +

In [325]:
movie_bd.groupby(['director'])['revenue'].sum().sort_values(ascending=False) #Сгруппируем данные по колонке director, выполним суммирование по параметру revenue и сортировку 

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 957, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [326]:
answers['14'] = '3. Robert Rodriguez'
# +

In [327]:
action_movies = movie_bd[movie_bd.genres.str.match("Action", na=False)] #Отфильтруем датасет по наличию Action колонке genres
collections.Counter(action_movies['director'].str.cat(sep = '|').split('|')).most_common(5) #подсчитаем значения, разделив по разделителю |

[('Robert Rodriguez', 7),
 ('Antoine Fuqua', 5),
 ('Andrzej Bartkowiak', 5),
 ('Tony Scott', 5),
 ('Brett Ratner', 4)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [328]:
answers['15'] = '3. Chris Hemsworth'
# +

In [329]:
movies_2012 = movie_bd.query('release_year == 2012') #Сформируем выборку фильмов по условию: значение в колонке release_year==2012
high_revenue_2012 = movies_2012[movies_2012.revenue > movies_2012.revenue.mean()] # сформируем выборку по revenue выше среднего
collections.Counter(high_revenue_2012['cast'].str.cat(sep = '|').split('|')).most_common(5) #подсчитаем значения, разделив по разделителю |

[('Chris Hemsworth', 2),
 ('Anne Hathaway', 2),
 ('Ralph Fiennes', 2),
 ('Charlize Theron', 2),
 ('Denis Leary', 2)]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [330]:
answers['16'] = '3. Matt Damon'
# +

In [331]:
high_budget = movie_bd[movie_bd.budget > movie_bd.budget.mean()] # сформируем выборку по budget выше среднего
collections.Counter(high_budget['cast'].str.cat(sep = '|').split('|')).most_common(5) #подсчитаем значения, разделив по разделителю |


[('Matt Damon', 18),
 ('Adam Sandler', 17),
 ('Angelina Jolie', 16),
 ('Tom Cruise', 15),
 ('Samuel L. Jackson', 15)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [332]:
answers['17'] = '3. Action'
# +

In [333]:
nicolas_cage = movie_bd[movie_bd.cast.str.match("Nicolas Cage", na=False)] #Отфильтруем датасет по наличию Nicolas Cage колонке cast
collections.Counter(nicolas_cage['genres'].str.cat(sep = '|').split('|')).most_common(5) #подсчитаем значения, разделив по разделителю |

[('Action', 16), ('Thriller', 15), ('Drama', 12), ('Crime', 9), ('Fantasy', 7)]

# 18. Самый убыточный фильм от Paramount Pictures

In [334]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'
# +

In [335]:
paramaunt = movie_bd[movie_bd.production_companies.str.contains('Paramount Pictures')] #Отфильтруем датасет по наличию Paramount Pictures колонке production_companies
paramaunt.sort_values('profit', ascending=True).head(1) # отсортируем полученные значения в обратном порядке по profit

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,name_length,production_companies_sep,overview_words,cast_pair
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,"[7, 19, 2002]",6.0,2002,-64831034,7,20,"[Paramount Pictures, Intermedia Films, Nationa...",23,"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ..."


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [336]:
answers['19'] = '5. 2015'
# +

In [337]:
movie_bd.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False) #сгруппируем данные по release_year и просуммируем по revenue


release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [338]:
answers['20'] = '4. 2014'
# +

In [339]:
wb_profit_year = movie_bd[movie_bd.production_companies.str.contains('Warner Bros')] #Отфильтруем датасет по наличию Paramount Pictures колонке production_companies
wb_profit_year.groupby(['release_year'])['profit'].sum().sort_values(ascending=False) #сгруппируем данные по release_year и просуммируем по profit

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [340]:
answers['21'] = '4. Сентябрь'
# +

In [341]:
movie_bd['month'].value_counts() # подсчитаем сколько раз встречается каждый месяц 

9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: month, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [342]:
answers['22'] = '2. 450'
# +

In [343]:
summer_movies = movie_bd.query('month in ["6","7","8"]') #сделаем выборку по колонке month
len(summer_movies.index) #Подсчитаем количество фильмов выборке

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [344]:
answers['23'] = '5. Peter Jackson'
# +

In [345]:
winter_movies = summer_movies = movie_bd.query('month in ["12","1","2"]') #сделаем выборку по колонке month
collections.Counter(winter_movies['director'].str.cat(sep = '|').split('|')).most_common(5) #подсчитаем значения, разделив по разделителю |

[('Peter Jackson', 7),
 ('Clint Eastwood', 6),
 ('Steven Soderbergh', 6),
 ('Shawn Levy', 4),
 ('Nancy Meyers', 4)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [346]:
answers['24'] = '5. Four By Two Productions'
# +

In [347]:
movies_sep = movie_bd.explode('production_companies_sep') #разделим каждый фильм по студиям production_companies_sep
movies_sep.groupby('production_companies_sep').name_length.mean().sort_values(ascending=False) #сгруппируем по среднему значению name_length


production_companies_sep
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Global Entertainment Group     2.0
Name: name_length, Length: 1771, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [348]:
answers['25'] = '5. Midnight Picture Show'
# +

In [349]:
overview_words = movie_bd.explode('production_companies_sep') #разделим каждый фильм по студиям production_companies_sep
overview_words.groupby(['production_companies_sep'])['overview_words'].mean().sort_values(ascending=False) #сгруппируем по среднему значению name_length


production_companies_sep
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
Heineken Branded Entertainment           161.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: overview_words, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [350]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
# +

In [351]:
movie_bd.sort_values(by='vote_average').tail(int(movie_bd.original_title.count()/100)).original_title #отсортируем фильмы по рейтингу и возьмем процент 

155                          The Theory of Everything
863             The Lord of the Rings: The Two Towers
1688                              There Will Be Blood
283                                        Mr. Nobody
1191                                 12 Years a Slave
872                                       The Pianist
128                                         Gone Girl
1800                                          Memento
119                           Guardians of the Galaxy
370                                         Inception
1183                          The Wolf of Wall Street
138                          The Grand Budapest Hotel
1081    The Lord of the Rings: The Return of the King
34                                               Room
118                                      Interstellar
9                                          Inside Out
125                                The Imitation Game
599                                   The Dark Knight
Name: original_title, dtype:

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

In [355]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
# +

In [359]:
together = movie_bd['cast_pair'].reset_index() #создадим новый дата фрейм
together['cast_pair'] = together['cast_pair'].apply(lambda s: list(itertools.combinations(s, 2))) #отберем парные значения
together = together.explode('cast_pair').reset_index() #создадим новый дата фрейм
collections.Counter(together['cast_pair']).most_common(10) #подсчитаем совпадения

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6),
 (('Adam Sandler', 'Kevin James'), 5),
 (('Hugh Jackman', 'Ian McKellen'), 5),
 (('Kristen Stewart', 'Robert Pattinson'), 5),
 (('Kristen Stewart', 'Taylor Lautner'), 5),
 (('Robert Pattinson', 'Taylor Lautner'), 5)]

# Submission

In [356]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '3. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '4. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '5. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [357]:
# и убедиться что ни чего не пропустил)
len(answers)

27